In [57]:
from utils import *
import mrf
import llm
from importlib import reload
from evaluate import *
import utils
import evaluate
reload(evaluate)
import json
import os
from tabulate import tabulate

In [35]:
# list all files in all subdirectories
def list_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".json"):
                yield os.path.join(root, file)

k = [i for i in list_files('./Experiments/trial/bin0')]
k

[]

In [36]:
def parse_results(path):
    js = json.load(open(path))
    ground = js['ground']
    cot = []
    zero = []
    mrf = []
    question = js['question']['Text']
    for key in js.keys():
        if key in ['question', 'ground']:
            continue
        cot.append(js[key]['cot']['probability'])
        zero.append(js[key]['zero']['probability'])
        mrf.append(js[key]['mrf']['probability'])
        query = js[key]['mrf']['query']
    return ground, cot, zero, mrf, question, query
        
def metrics(ground, res):
    return evaluate.total_variation_distance(ground, res).round(3)

def metrics2(ground, res):
    return evaluate.kl_divergence(ground, res).round(3)



In [37]:

for j in list_files('./Experiments/constraints/bin1'):
    ground, cot, zero, mrf, question, query = parse_results(j)
    if len(cot) == 0:
        continue
    print(j, question)
    print(query)
    cot = np.array(cot[0])
    zero = np.array(zero[0])
    mrf = np.array(mrf[0])
    ground = np.array(ground)
    table = [
        ["Method", "total_variation_distance", "dist"],
        ["CoT", metrics(ground, cot), cot.round(2)],
        ["mrf", metrics(ground, mrf), mrf.round(2)],
        ["ground", metrics(ground, ground), ground.round(2)],
        ["zero", metrics(ground, zero), zero.round(2)],
    ]
    table2 = [
        ["Method", "kl_divergence", "dist"],
        ["CoT", metrics2(ground, cot), cot.round(2)],
        ["mrf", metrics2(ground, mrf), mrf.round(2)],
        ["ground", metrics2(ground, ground), ground.round(2)],
        ["zero", metrics2(ground, zero), zero.round(2)],
    ]
    print(tabulate(table))
    print("#################################################")

In [38]:
for j in list_files('./Experiments/trial/bin0'):
    ground, cot, zero, mrf, question = parse_results(j)
    print(j, question)

In [39]:
for j in list_files('./Dataset/bin0'):
    if j.endswith('q.json'):
        file = json.load(open(j))
        if 'Text' in file.keys():
            print(file['Text'])
        else:
            print('no text')

What is the nightly price of an Airbnb listing in Rochester, NY with 2 bedrooms?
What is the price of an Airbnb listing in Newark, NJ for the months of May to August?
What is the price of an Airbnb listing with 2 beds in Columbus, OH?
What is the nightly price of an Airbnb listing in Broward County, FL that accommodates 2 guests?
What is the price of an Airbnb listing in San Diego, CA with 0 to 1 bathroom?
What is the price of an Airbnb listing in Cambridge, MA for a host who is not a superhost?
What is the price of an Airbnb listing hosted by a superhost in Santa Clara County, CA?
What is the price of an Airbnb listing in Clark County, NV with no review scores available?
What is the price of an Airbnb listing in Dallas, TX with a review scores rating of 4.9 or higher?
What is the price of an Airbnb listing in Los Angeles, CA that accommodates 4 or more guests per night?
What is the price of an Airbnb listing in Salem, OR, hosted by someone who has been hosting for 1-5 years?
What is t

In [40]:
k = [i for i in list_files('./Experiments/trial/bin0')]
b = ['./Experiments/trial/bin0/LosAngeles/LosAngeles_q.json']

In [41]:
import mrf
import importlib
importlib.reload(mrf)

k = [i for i in list_files('./Experiments/trial/bin0')]


for j in b[:1]:
    print(j)
    js = json.load(open(j))
    for key in js.keys():
        if key in ['question', 'ground']:
            continue
        #print(key)
        query = js[key]['mrf']['query']
        query = json.loads(query)
        query['Queries'][0]['Condition'] = [query['Queries'][0]['Condition'][0]]
        print(query)
        test = mrf.Brute(verbose=True)
        test.from_json(vars=js[key]['mrf']['json']['Variables'], constraints=pp)
        #test.from_json(pp)
        test.update(10000)
        test.query(query=query)
        break


./Experiments/trial/bin0/LosAngeles/LosAngeles_q.json


FileNotFoundError: [Errno 2] No such file or directory: './Experiments/trial/bin0/LosAngeles/LosAngeles_q.json'

In [ ]:
reload(utils)
p = './Experiments/35/'

for j in os.listdir(p):
    # check if is folder
    if os.path.isdir(p+j):
        dat = None
        for file in os.listdir(p+j):
            if file.endswith('.csv'):
                dat = utils.DatasetQ(p+j+'/'+file)
            elif file.endswith('.json'):
                constraints = json.load(open(p+j+'/'+file))
        table = [["Constraint", "Ground", "Zero", "Joint", "Change", "Zero dist", "Joint dist"],]
        tot_change = 0
        #print(constraints['Variables'])
        for cons in tmpp['Constraints']:
            er = False
            ground = dat.marg(cons)
            pred = cons['Probability']
            if np.isnan(ground).any():
                print("Error: nan in ground", cons['Question'])
                print("#################################################")
                continue
            query = {}
            query['Queries'] = [{'Condition': cons['Condition'], 'Target': cons['Target']}]
            after = mrfff.query(query)
            ground = np.array(ground)
            zero = np.array(pred)
            after = np.array(after)
            tar_name = cons['Target'][0]['Name']
            cons_val = '[]'
            if len(cons['Condition']) > 0:
                cons_val = cons['Condition'][0]['Value']
            try:
                zero_metric = metrics2(ground, zero)
            except:
                er = True
                zero_metric = float('nan')
            after_metric = metrics2(ground, after)
            table.append([f"{tar_name}|{cons_val}", str(ground.round(2)), str(zero.round(2)), str(after.round(2)), f"{(after_metric-zero_metric):+.3f}", zero_metric, after_metric])
            if not er:
                tot_change += after_metric-zero_metric
        print(tabulate(table))
        print("#################################################")
        print(f"Total change: {tot_change}")


.DS_Store
BrowardCounty
LosAngeles


In [ ]:
dat = None
pa = './Experiments/35/BrowardCounty'
for file in os.listdir(pa):
    if file.endswith('.csv'):
        dat = utils.DatasetQ(pa+'/'+file)
    elif file.endswith('.json'):
        constraints = json.load(open(pa+'/'+file))
table = [["Constraint", "Ground", "Zero", "Joint", "Change", "Zero dist", "Joint dist"],]
tot_change = 0
#con = remove_vars(constraints, ['Month'])
#print(constraints['Variables'])
for cons in constraints['Constraints']:
    er = False
    ground = dat.marg(cons)
    pred = cons['Probability']
    if np.isnan(ground).any():
        print("Error: nan in ground", cons)
        print("#################################################")
        continue
    query = {}
    query['Queries'] = [{'Condition': cons['Condition'], 'Target': cons['Target']}]
    after = model.query(query)
    ground = np.array(ground)
    zero = np.array(pred)
    after = np.array(after)
    tar_name = cons['Target'][0]['Name']
    cons_val = '[]'
    if len(cons['Condition']) > 0:
        cons_val = cons['Condition'][0]['Value']
    try:
        zero_metric = metrics(ground, zero)
    except:
        er = True
        zero_metric = float('nan')
    after_metric = metrics(ground, after)
    table.append([f"{tar_name}|{cons_val}", str(ground.round(2)), str(zero.round(2)), str(after.round(2)), f"{(after_metric-zero_metric):+.3f}", zero_metric, after_metric])
    if not er:
        tot_change += after_metric-zero_metric
print(tabulate(table))
print("#################################################")
print(f"Total change: {tot_change}")

FileNotFoundError: [Errno 2] No such file or directory: './Experiments/35/BrowardCounty'

In [46]:
dat = None
pa = './Experiments3/LosAngeles'
dat = utils.DatasetQ('./data/LosAngeles.csv')
for file in os.listdir(pa):
    if file.endswith('.csv'):
        dat = utils.DatasetQ(pa+'/'+file)
    elif file.endswith('agents2.json'):
        constraints = json.load(open(pa+'/'+file))


import mrf
reload(mrf)

model = mrf.Brute(verbose=True)
ret = model.from_json(constraints['schema'])
model.set_w0(0.4)
model.update(10000)

-----------------------------------------------
Finished building
Features:  113
Constraints:  112
-----------------------------------------------


Training Progress:   0%|          | 10/10000 [00:00<01:43, 96.93it/s]

Entropy:  -2.23779559135437
Loss:  tensor(7.3527, grad_fn=<AddBackward0>) torch.return_types.topk(
values=tensor([0.7935, 0.7415, 0.6387, 0.6348, 0.5360], grad_fn=<TopkBackward0>),
indices=tensor([27, 41, 11, 94, 78]))


Training Progress:  20%|██        | 2005/10000 [00:20<01:15, 106.44it/s]

Entropy:  -3.4043216705322266
Loss:  tensor(-3.1953, grad_fn=<AddBackward0>) torch.return_types.topk(
values=tensor([0.1373, 0.1026, 0.0898, 0.0892, 0.0743], grad_fn=<TopkBackward0>),
indices=tensor([  0,  23, 100,   4,  22]))


...

Entropy:  -3.4042255878448486
Loss:  tensor(-3.1953, grad_fn=<AddBackward0>) torch.return_types.topk(
values=tensor([0.1373, 0.1026, 0.0898, 0.0891, 0.0744], grad_fn=<TopkBackward0>),
indices=tensor([  0,  23, 100,   4,  22]))


Training Progress:  80%|████████  | 8022/10000 [01:17<00:20, 95.97it/s] 

Entropy:  -3.4042255878448486
Loss:  tensor(-3.1953, grad_fn=<AddBackward0>) torch.return_types.topk(
values=tensor([0.1373, 0.1026, 0.0898, 0.0891, 0.0744], grad_fn=<TopkBackward0>),
indices=tensor([  0,  23, 100,   4,  22]))


Training Progress: 100%|██████████| 10000/10000 [01:36<00:00, 104.10it/s]

Entropy:  -3.4042255878448486
Loss:  tensor(-3.1953, grad_fn=<AddBackward0>) torch.return_types.topk(
values=tensor([0.1373, 0.1026, 0.0898, 0.0891, 0.0744], grad_fn=<TopkBackward0>),
indices=tensor([  0,  23, 100,   4,  22]))


In [ ]:
dat = utils.DatasetQ('./data/LosAngeles.csv')

In [ ]:
prob_file = json.load(open('./Experiments3/LosAngeles/prob_cache.json'))

In [ ]:
var_dict = {}
for var in constraints['schema']['Variables']:
    var_dict[var['Name']] = var['Value']

In [54]:
modell = mrf.Brute(verbose=True)
ret = modell.from_json(schema)
modell.update(10000)

-----------------------------------------------
Finished building
Features:  113
Constraints:  112
-----------------------------------------------


Training Progress:   0%|          | 12/10000 [00:00<01:27, 114.42it/s]

Loss:  tensor(10.1424, grad_fn=<SumBackward0>) torch.return_types.topk(
values=tensor([0.8624, 0.7190, 0.6365, 0.6014, 0.5197], grad_fn=<TopkBackward0>),
indices=tensor([108,  77,   4,  74,  27]))


Training Progress:  20%|██        | 2013/10000 [00:18<01:12, 110.84it/s]

Loss:  tensor(0.0103, grad_fn=<SumBackward0>) torch.return_types.topk(
values=tensor([0.0474, 0.0163, 0.0150, 0.0138, 0.0138], grad_fn=<TopkBackward0>),
indices=tensor([24, 81, 21, 22, 23]))


...

Loss:  tensor(0.0077, grad_fn=<SumBackward0>) torch.return_types.topk(
values=tensor([0.0449, 0.0188, 0.0170, 0.0124, 0.0120], grad_fn=<TopkBackward0>),
indices=tensor([24, 96, 21, 22, 23]))


Training Progress:  80%|████████  | 8016/10000 [01:09<00:17, 111.16it/s]

Loss:  tensor(0.0076, grad_fn=<SumBackward0>) torch.return_types.topk(
values=tensor([0.0446, 0.0197, 0.0172, 0.0123, 0.0117], grad_fn=<TopkBackward0>),
indices=tensor([24, 96, 21, 22, 23]))


Training Progress: 100%|██████████| 10000/10000 [01:26<00:00, 115.97it/s]

Loss:  tensor(0.0075, grad_fn=<SumBackward0>) torch.return_types.topk(
values=tensor([0.0444, 0.0202, 0.0173, 0.0123, 0.0115], grad_fn=<TopkBackward0>),
indices=tensor([24, 96, 21, 22, 23]))


In [51]:
schema

{'Question': {'Target': [{'Name': 'Price'}],
  'Text': 'What is the nightly price of an Airbnb listing in Los Angeles, CA?'},
 'Variables': [{'Name': 'Number of Bedrooms',
   'Value': ['0/1 bedroom', '2 bedrooms', '3 bedrooms', '4+ bedrooms']},
  {'Name': 'Number of Bathrooms',
   'Value': ['0/1 bathroom', '2 bathrooms', '3+ bathrooms']},
  {'Name': 'Host Experience',
   'Value': ['<1 year', '1-5 years', '5-10 years', '10+ years']},
  {'Name': 'Superhost', 'Value': ['Yes', 'No']},
  {'Name': 'Room Type',
   'Value': ['Entire home/apt', 'Private room', 'Shared/Hotel room']},
  {'Name': 'Total Beds', 'Value': ['0/1 bed', '2 beds', '3 beds', '4+ beds']},
  {'Name': 'Review Score', 'Value': ['<4.3', '4.3-4.8', '4.9+']},
  {'Name': 'Price',
   'Value': ['<$50', '$51-$100', '$101-$200', '$201-$500', '$500+']}],
 'Constraints': [{'Target': [{'Name': 'Superhost', 'Value': ['Yes', 'No']}],
   'Condition': [],
   'Probability': [0.3297555944642986, 0.6702444055357014]},
  {'Target': [{'Name': 'R

In [50]:
schema = constraints['schema']
schema['Constraints'] = golds

In [49]:
golds

[{'Target': [{'Name': 'Superhost', 'Value': ['Yes', 'No']}],
  'Condition': [],
  'Probability': [0.3297555944642986, 0.6702444055357014]},
 {'Target': [{'Name': 'Room Type',
    'Value': ['Entire home/apt', 'Private room', 'Shared/Hotel room']}],
  'Condition': [],
  'Probability': [0.7294291944800493,
   0.2522209090051548,
   0.018349896514795887]},
 {'Target': [{'Name': 'Number of Bathrooms',
    'Value': ['0/1 bathroom', '2 bathrooms', '3+ bathrooms']}],
  'Condition': [],
  'Probability': [0.6677771115108713,
   0.22307171252803873,
   0.10915117596109002]},
 {'Target': [{'Name': 'Host Experience',
    'Value': ['<1 year', '1-5 years', '5-10 years', '10+ years']}],
  'Condition': [],
  'Probability': [0.07762615359856158,
   0.26707436860769046,
   0.5176390283966992,
   0.13766044939704877]},
 {'Target': [{'Name': 'Price',
    'Value': ['<$50', '$51-$100', '$101-$200', '$201-$500', '$500+']}],
  'Condition': [],
  'Probability': [0.0687830380144335,
   0.2384261547676508,
   0.3

In [56]:
golds = []

table = [["Constraint", "Ground", "Zero", "Joint", "Change", "Zero dist", "Joint dist"],]
tot_change = 0
cnt = 0

for i, cons in enumerate(constraints['schema']['Constraints']):
    er = False
    ground = dat.marg(cons, target_values=var_dict[cons['Target'][0]['Name']])
    pred = cons['Probability']
    if np.isnan(ground).any():
        print("Error: nan in ground", cons)
        print("#################################################")
        continue
    query = {}
    query['Queries'] = [{'Target': cons['Target'], 'Condition': cons['Condition']}]
    after = model.query(query)
    zero = np.array(modell.query(query))

    ground = np.array(ground)
    #golds.append({'Target': cons['Target'], 'Condition': cons['Condition'], 'Probability': ground.tolist()})
    #zero = np.array(j['Probability'])
    after = np.array(after)
    tar_name = cons['Target'][0]['Name']
    cons_val = '[]'
    if len(cons['Condition']) > 0:
        cons_val = cons['Condition'][0]['Value']
    try:
        zero_metric = metrics(ground, zero)
    except:
        er = True
        zero_metric = float('nan')
    after_metric = metrics(ground, after)
    table.append([f"{tar_name}|{cons_val}", str(ground.round(2)), str(zero.round(2)), str(after.round(2)), f"{(after_metric-zero_metric):+.3f}", zero_metric, after_metric])
    if not er:
        tot_change += after_metric-zero_metric
        cnt += 1
print(tabulate(table))
print("#################################################")
print(f"Average change: {tot_change/cnt}")

-----------------------------------  --------------------------  --------------------------  --------------------------  ------  ---------  ----------
Constraint                           Ground                      Zero                        Joint                       Change  Zero dist  Joint dist
Superhost|[]                         [0.33 0.67]                 [0.33 0.67]                 [0.29 0.71]                 +0.042  0.0        0.042
Room Type|[]                         [0.73 0.25 0.02]            [0.73 0.25 0.01]            [0.51 0.3  0.19]            +0.219  0.004      0.223
Number of Bathrooms|[]               [0.67 0.22 0.11]            [0.65 0.23 0.12]            [0.47 0.32 0.2 ]            +0.177  0.018      0.195
Host Experience|[]                   [0.08 0.27 0.52 0.14]       [0.08 0.27 0.52 0.14]       [0.27 0.39 0.23 0.1 ]       +0.319  0.001      0.32
Price|[]                             [0.07 0.24 0.36 0.24 0.09]  [0.07 0.24 0.36 0.24 0.08]  [0.08 0.18 0.34 0.28 0

In [47]:
table = [["Constraint", "Ground", "Zero", "Joint", "Change", "Zero dist", "Joint dist"],]
tot_change = 0
cnt = 0

for i, cons in enumerate(constraints['schema']['Constraints']):
    er = False
    ground = dat.marg(cons, target_values=var_dict[cons['Target'][0]['Name']])
    pred = cons['Probability']
    if np.isnan(ground).any():
        print("Error: nan in ground", cons)
        print("#################################################")
        continue
    query = {}
    query['Queries'] = [{'Target': cons['Target'], 'Condition': cons['Condition']}]
    after = model.query(query)
    for j in prob_file[json.dumps(query['Queries'][0])]:
        ground = np.array(ground)
        zero = np.array(j['Probability'])
        after = np.array(after)
        tar_name = cons['Target'][0]['Name']
        cons_val = '[]'
        if len(cons['Condition']) > 0:
            cons_val = cons['Condition'][0]['Value']
        try:
            zero_metric = metrics(ground, zero)
        except:
            er = True
            zero_metric = float('nan')
        after_metric = metrics(ground, after)
        table.append([f"{tar_name}|{cons_val}", str(ground.round(2)), str(zero.round(2)), str(after.round(2)), f"{(after_metric-zero_metric):+.3f}", zero_metric, after_metric])
    if not er:
        tot_change += after_metric-zero_metric
        cnt += 1
print(tabulate(table))
print("#################################################")
print(f"Average change: {tot_change/cnt}")

-----------------------------------  --------------------------  --------------------------  --------------------------  ------  ---------  ----------
Constraint                           Ground                      Zero                        Joint                       Change  Zero dist  Joint dist
Superhost|[]                         [0.33 0.67]                 [0.25 0.75]                 [0.29 0.71]                 -0.038  0.08       0.042
Superhost|[]                         [0.33 0.67]                 [0.2 0.8]                   [0.29 0.71]                 -0.088  0.13       0.042
Superhost|[]                         [0.33 0.67]                 [0.18 0.82]                 [0.29 0.71]                 -0.108  0.15       0.042
Superhost|[]                         [0.33 0.67]                 [0.25 0.75]                 [0.29 0.71]                 -0.038  0.08       0.042
Superhost|[]                         [0.33 0.67]                 [0.15 0.85]                 [0.29 0.71]          

In [ ]:
prob_file.keys()

dict_keys(['{"Target": [{"Name": "Superhost", "Value": ["Yes", "No"]}], "Condition": []}', '{"Target": [{"Name": "Number of Bathrooms", "Value": ["0/1 bathroom", "2 bathrooms", "3+ bathrooms"]}], "Condition": []}', '{"Target": [{"Name": "Host Experience", "Value": ["<1 year", "1-5 years", "5-10 years", "10+ years"]}], "Condition": []}', '{"Target": [{"Name": "Number of Bedrooms", "Value": ["0/1 bedroom", "2 bedrooms", "3 bedrooms", "4+ bedrooms"]}], "Condition": []}', '{"Target": [{"Name": "Review Score", "Value": ["<4.3", "4.3-4.8", "4.9+"]}], "Condition": []}', '{"Target": [{"Name": "Price", "Value": ["<$50", "$51-$100", "$101-$200", "$201-$500", "$500+"]}], "Condition": []}', '{"Target": [{"Name": "Total Beds", "Value": ["0/1 bed", "2 beds", "3 beds", "4+ beds"]}], "Condition": []}', '{"Target": [{"Name": "Room Type", "Value": ["Entire home/apt", "Private room", "Shared/Hotel room"]}], "Condition": []}', '{"Target": [{"Name": "Review Score", "Value": ["<4.3", "4.3-4.8", "4.9+"]}], "

In [ ]:
tmp = con.copy()
tmp = remove_vars(tmp, [], "Price")
tmp

{'Variables': [{'Name': 'Number of Bedrooms',
   'Value': ['0/1 bedroom', '2 bedrooms', '3 bedrooms', '4+ bedrooms']},
  {'Name': 'Number of Bathrooms',
   'Value': ['0/1 bathroom', '2 bathrooms', '3+ bathrooms']},
  {'Name': 'Host Experience',
   'Value': ['<1 year', '1-5 years', '5-10 years', '10+ years']},
  {'Name': 'Superhost', 'Value': ['Yes', 'No']},
  {'Name': 'Room Type',
   'Value': ['Entire home/apt', 'Private room', 'Shared room']},
  {'Name': 'Review Score', 'Value': ['<4.3', '4.3-4.8', '4.9+']},
  {'Name': 'Price',
   'Value': ['<$50', '$51-$100', '$101-$200', '$201-$500', '$500+']}],
 'Constraints': [{'Target': [{'Name': 'Number of Bedrooms',
     'Value': ['0/1 bedroom', '2 bedrooms', '3 bedrooms', '4+ bedrooms']}],
   'Probability': [0.3, 0.4, 0.2, 0.1],
   'Condition': []},
  {'Target': [{'Name': 'Number of Bathrooms',
     'Value': ['0/1 bathroom', '2 bathrooms', '3+ bathrooms']}],
   'Probability': [0.4, 0.35, 0.25],
   'Condition': []},
  {'Target': [{'Name': 'Host

In [ ]:
con['Constraints']

[{'Target': [{'Name': 'Number of Bedrooms',
    'Value': ['0/1 bedroom', '2 bedrooms', '3 bedrooms', '4+ bedrooms']}],
  'Probability': [0.3, 0.4, 0.2, 0.1],
  'Condition': []},
 {'Target': [{'Name': 'Number of Bathrooms',
    'Value': ['0/1 bathroom', '2 bathrooms', '3+ bathrooms']}],
  'Probability': [0.5, 0.3, 0.2],
  'Condition': []},
 {'Target': [{'Name': 'Host Experience',
    'Value': ['<1 year', '1-5 years', '5-10 years', '10+ years']}],
  'Probability': [0.2727, 0.4545, 0.1818, 0.0909],
  'Condition': []},
 {'Target': [{'Name': 'Superhost', 'Value': ['Yes', 'No']}],
  'Probability': [0.3, 0.7],
  'Condition': []},
 {'Target': [{'Name': 'Room Type',
    'Value': ['Entire home/apt', 'Private room', 'Shared room']}],
  'Probability': [0.6, 0.35, 0.05],
  'Condition': []},
 {'Target': [{'Name': 'Review Score', 'Value': ['<4.3', '4.3-4.8', '4.9+']}],
  'Probability': [0.15, 0.6, 0.25],
  'Condition': []},
 {'Target': [{'Name': 'Price',
    'Value': ['<$50', '$51-$100', '$101-$200',

In [ ]:
def remove_vars(constraints, vars, one=''):
    ret = {}
    ret['Variables'] = []
    ret['Constraints'] = []
    check = set()
    for var in constraints['Variables']:
        if var['Name'] not in vars:
            ret['Variables'].append(var)
    for cons in constraints['Constraints']:
        if cons is None:
            print("None")
            continue
        if cons['Target'][0]['Name'] not in vars and (len(cons['Condition']) == 0 or (cons['Condition'][0]['Name'] not in vars)):
            if len(cons['Condition']) != 0 or cons['Target'][0]['Name'] != one:
                tt = {}
                tt['Target'] = cons['Target']
                tt['Probability'] = cons['Probability']
                #tt['Question'] = cons['Question']
                tt['Condition'] = []
                for cond in cons['Condition']:
                    if cond['Name'] == 'City':
                        continue
                    tt['Condition'].append(cond)
                ret['Constraints'].append(tt)
    return ret